In [ ]:
using Logging
logger = ConsoleLogger(stdout)
# debuglogger = ConsoleLogger(stderr, Logging.Debug)
global_logger(logger)
using JLD

using Random
rng = MersenneTwister(1234)
import Dates

In [ ]:
using Plots
pyplot()

In [ ]:
include("utils/payoff_functions.jl")
using .PayoffFunctions
include("weightonly/plot_game_findom.jl")
using .PlotGameFindomUtils
include("weightonly/plot_results_findom.jl")
using .PlotResFindom
include("weightonly/MP_bilin_game.jl")
using .MPBilinGame
include("weightonly/game_theory_findom.jl")
using .GameTheoryFindomUtils

In [ ]:
ts = Dates.now()

alpha = 1
scaling = 2

## algo parameters
T = Int(floor(50000 / (alpha^scaling)))
mx = 14 # size of strategy space for player x
my = 12
eta0_wx = 2e0 * alpha # initial stepsize
eta0_wy = 2e0 * alpha
extrasteps = 2 # extrasteps=1: Mirror Descent-Ascent, extrasteps=2: Mirror Prox

## random payoff matrix parameters
sigma = 1.0

## plotting parameters
evalNIevery = Int(floor(T/50))
evalgap0every = Int(floor(T/50))
evaldiv0every = Int(floor(T/50))
plotiterevery = Int(floor(T/50))

## logging
ts_fsfriendly = Dates.format(ts, "yyyy-mm-ddTHHMMSS") # filesystem-friendly string for ts
resultdir = mkpath("results/weightonly__gaussiansigma$(sigma)__mx$(mx)y$(my)__extrasteps$(extrasteps)__$(ts_fsfriendly)")

logfile = "$resultdir/log.txt"
touch(logfile)
open(logfile, "a") do f
    write(f, "T=$T\n")
    write(f, "mx=$(mx)\n")
    write(f, "my=$(my)\n")
    write(f, "eta0_wx=$(eta0_wx)\n")
    write(f, "eta0_wy=$(eta0_wy)\n")
    write(f, "extrasteps=$(extrasteps)\n")
    write(f, "sigma=$(sigma)\n")
end


In [ ]:
## instantiate game
Random.seed!(rng, 1234)
gmat = random_gaussian_matrix(mx, my; rng=rng, sigma=sigma)

open(logfile, "a") do f
    write(f, "minimum(gmat)=$(minimum(gmat))\n")
    write(f, "maximum(gmat)=$(maximum(gmat))\n")
end
heatmap(transpose(gmat))

In [ ]:
## solve game
#==============#
copies_wx, copies_wy = run_MP(gmat, T, eta0_wx, eta0_wy; extrasteps=extrasteps)
avg_wx, avg_wy = avg_iter(copies_wx, copies_wy)
save("$resultdir/iterates.jld", "copies_wx", copies_wx, "copies_wy", copies_wy)

In [ ]:
# Use the last iterate as reference point (estimate of the true MNE)
wx0, wy0 = copies_wx[:,T+1], copies_wy[:,T+1]
nierr0 = NI_err(gmat, wx0, wy0)
gval0 = payoff(gmat, wx0, wy0)
open(logfile, "a") do f
    write(f, "Using last iterate as reference point (wx0, wy0) with NI error $(nierr0)\n")
    write(f, "(For comparison: max(gmat)-min(gmat)=$(maximum(gmat) - minimum(gmat)))\n")
    write(f, "Payoff at reference point: gval0=$(gval0)\n")
end
nierr0, gval0

In [ ]:
## compute and plot optimality metrics
#==============#
### compute and plot NI error of iterates
plt_NI, plt_NI_log, plt_NI_log_log, nierrs = plot_NI_err(gmat, T, copies_wx, copies_wy, avg_wx, avg_wy, 
    resultdir, logfile, evalNIevery)
save("$resultdir/nierrs__every$(evalNIevery).jld", "nierrs", nierrs)
plt_NI_log

In [ ]:
### compute and plot KL-divergence to reference point of iterates
plt_div0, plt_div0_log, div0s = plot_div_to_ref(gmat, T, copies_wx, copies_wy, avg_wx, avg_wy, 
    wx0, wy0, 
    resultdir, logfile, evaldiv0every)
plt_div0_log

In [ ]:
## plot first variations at reference point
#==============#
plt_firstvar_x, plt_firstvar_y = plot_firstvars(gmat, wx0, wy0, gval0,
    resultdir)
plt_firstvar_x

In [ ]:
## plot iterations
#==============#
pltt = plot_iter_findom(gmat, wx0, wy0)
fn = "$resultdir/last_iter.png"
savefig(pltt, fn)
pltt

In [ ]:
plot_iters(gmat, T, copies_wx, copies_wy, avg_wx, avg_wy, 
    resultdir,
    plotiterevery)